In [1]:
import pandas as pd  #importing all the important packages
import numpy as np
import matplotlib.pyplot as plt #currently not used
import seaborn as sns #currently not used
import pickle
pickle_in = open('pokemon.pickle','rb')
df = pickle.load(pickle_in)
df.columns = df.columns.str.upper().str.replace('_', '') 
df['TYPE2'] = df['TYPE2'].replace(to_replace=[None], value=np.nan, inplace=True)
df['TYPE2'].fillna(df['TYPE1'], inplace=True)
df = df.set_index('NAME')
df=df.drop(['ID'],axis=1)

# Damage multiplier for type matchups

The function accepts two types, the first type is the attacking move type and the second is the defenders type (if the defender has multiple types this function should be run multiple times and multiplied together).

In [2]:
def type_weakness(type1,type2):
    if type1 == 'normal':
        if type2 in ['rock' ,'steel']:
            return 0.5
        elif type2 == 'ghost':
            return 0
    if type1 == 'fire':
        if type2 in ['water','fire','rock', 'dragon']:
            return 0.5
        elif type2 in ['grass', 'ice', 'bug','steel']:
            return 2
    if type1 == 'water':
        if type2 in ['water', 'grass', 'dragon']:
            return 0.5
        if type2 in ['fire', 'ground', 'rock']:
            return 2
    if type1 == 'grass':
        if type2 in ['fire', 'grass', 'poison' , 'dragon', 'steel', 'bug', 'flying']:
            return 0.5
        if type2 in ['water' , 'ground' ,'rock']:
            return 2
    if type1 == 'electric':
        if type2 == 'ground':
            return 0
        if type2 in ['grass', 'electric', 'dragon']:
            return 0.5
        if type2 in ['water','flying']:
            return 2
    if type1 == 'ice':
        if type2 in ['fire', 'water', 'ice', 'steel']:
            return 0.5
        if type2 in ['grass','flying','ground', 'dragon']:
            return 2
    if type1 == 'fighting':
        if type2 == 'ghost':
            return 0
        if type2 in ['poison', 'flying', 'psychic', 'bug', 'fairy']:
            return 0.5
        if type2 in ['normal', 'ice','rock', 'dark', 'steel']:
            return 2
    if type1 == 'poison':
        if type2 == 'steel':
            return 0
        if type2 in ['poison', 'ground', 'rock', 'ghost']:
            return 0.5
        if type2 in['fairy', 'grass']:
            return 2
    if type1 == 'ground':
        if type2 == 'flying':
            return 0
        if type2 in['grass','bug']:
            return 0.5
        if type2 in ['fire', 'electric','rock', 'steel', 'poison']:
            return 2
    if type1 == 'flying':
        if type2 in['electric','steel', 'rock']:
            return 0.5
        if type2 in ['grass', 'fighting', 'bug']:
            return 2
    if type1 == 'psychic':
        if type2 == 'dark':
            return 0
        if type2 in['steel','psychic']:
            return 0.5
        if type2 in ['fighting', 'poison']:
            return 2
    if type1 == 'bug':
        if type2 in ['fire', 'fighting', 'poison', 'flying', 'ghost' ,'steel', 'fairy']:
            return 0.5
        if type2 in ['dark','psychic','ghost']:
            return 2
    if type1 == 'rock':
        if type2 in ['steel', 'fighting', 'ground']:
            return 0.5
        if type2 in ['bug' ,'flying','ice','fire']:
            return 2
    if type1 == 'ghost':
        if type2 == 'normal':
            return 0
        if type2 == 'dark':
            return 0.5
        if type2 in ['psychic','ghost']:
            return 2
    if type1 == 'dragon':
        if type2 == 'fairy':
            return 0
        if type2 == 'steel':
            return 0.5
        if type2 == 'dragon':
            return 2
    if type1 == 'dark':
        if type2 in ['fighting','dark','fairy']:
            return 0.5
        if type2 in ['ghost','psychic']:
            return 2
    if type1 == 'steel':
        if type2 in ['steel','fire','water','electric']:
            return 0.5
        if type2 in ['ice','rock','fairy']:
            return 2
    if type1 == 'fairy':
        if type2 in ['fire', 'poison', 'steel']:
            return 0.5
        if type2 in ['dragon', 'fighting', 'dark']:
            return 2
    return 1

# Damage Calculations

In [3]:
def gen1_dmg(attack, defence, lvl, crit, power, movetype, physical,stab,deftype1,deftype2):
    #initial dmg
    dmg = 2*lvl
    #if crits
    if crit == True:
        dmg = 2*dmg
    #extras
    dmg = dmg/5 + 2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*attack/defence
    #extras
    dmg =dmg/50+2
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)*type_weakness(movetype,deftype2)
    return [int(dmg*(217/255)),int(dmg)]

In [4]:
def gen2_dmg(attack, defence, lvl, crit, power, movetype, physical,stab,deftype1,deftype2,item,badge,weather):
    #initial dmg
    dmg = 2*lvl/5+2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*attack/defence
    #extras
    dmg =dmg/50
    #if crits
    if crit == True:
        dmg = 2*dmg
    #item multiplier
    dmg = dmg*item
    #extras
    dmg = dmg+2
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #badge
    dmg = dmg*badge
    #weather
    if weather == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)*type_weakness(movetype,deftype2)
    
    return [int(dmg*(217/255)),int(dmg)]

In [5]:
def gen3_dmg(attack, defence, lvl, crit, power, movetype, physical,stab,deftype1,deftype2,item,weather,burn,screen,double,targetall,FF,WBR,HH,charge):
    #initial dmg
    dmg = 2*lvl/5+2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*attack/defence
    #extras
    dmg =dmg/50
    #burned (and not guts)
    if burn == True:
        dmg = dmg/2
    #screen
    if screen == True:
        if double == True:
            dmg = dmg*2/3
        else:
            dmg = dmg/2
    #in double battle dmg split between targets
    if double == True and targetall == False:
        dmg = dmg/2
    #flash fire
    if FF == True:
        dmg = dmg*1.5
    #weather
    if weather == True:
        dmg = dmg*1.5
    #extras
    dmg = dmg+2
    #item multiplier (assuming this goes here)
    dmg = dmg*item
    #extras
    dmg = dmg+2
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)*type_weakness(movetype,deftype2)
    #if crits
    if crit == True:
        dmg = 2*dmg
    #weatherball revenge (only in clear weather)
    if WBR == True:
        dmg = dmg*2
    #helping hand
    if HH == True:
        dmg = dmg*1.5
    #electric type charge effect
    if charge == True:
        dmg =dmg*2
    return [int(dmg*(85/100)),int(dmg)]

In [6]:
def gen4_dmg(attack, defence, lvl, crit, power, movetype, physical,stab,deftype1,deftype2,item,weather,burn,screen,double,targetall,FF,WBR,HH,charge,other):
    #initial dmg
    dmg = 2*lvl/5+2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*attack/defence
    #extras
    dmg =dmg/50
    #burned (and not guts)
    if burn == True:
        dmg = dmg/2
    #screen
    if screen == True:
        if double == True:
            dmg = dmg*2/3
        else:
            dmg = dmg/2
    #in double battle dmg split between targets
    if double == True and targetall == False:
        dmg = dmg/2
    #flash fire
    if FF == True:
        dmg = dmg*1.5
    #weather
    if weather == True:
        dmg = dmg*1.5
    #extras
    dmg = dmg+2
    #item multiplier (assuming this goes here)
    dmg = dmg*item
    #extras
    dmg = dmg+2
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)*type_weakness(movetype,deftype2)
    #if crits
    if crit == True:
        dmg = 2*dmg
    #weatherball revenge (only in clear weather)
    if WBR == True:
        dmg = dmg*2
    #helping hand
    if HH == True:
        dmg = dmg*1.5
    #electric type charge effect
    if charge == True:
        dmg =dmg*2
    #any other abilities and effects
    dmg = dmg*other
    return [int(dmg*(85/100)),int(dmg)]

In [7]:
def dmg(ratio,lvl,crit,power,movetype,stab,deftype1,deftype2,other):
    #initial dmg
    dmg = 2*lvl/5+2
    #move power
    dmg = dmg*power
    #Physical or Special dmg
    dmg = dmg*ratio
    #extras
    dmg =dmg/50+2
    #burned (and not guts)
    #if burn == True:
    #    dmg = dmg/2
    #screen
    #if screen == True:
    #    if double == True:
    #        dmg = dmg*2/3
    #    else:
    #        dmg = dmg/2
    #in double battle dmg split between targets
    #if double == True and targetall == False:
    #    dmg = dmg/2
    #flash fire
    #if FF == True:
    #    dmg = dmg*1.5
    #weather
    #if weather == True:
    #    dmg = dmg*1.5
    #item multiplier (assuming this goes here)
    #dmg = dmg*item
    #STAB dmg
    if stab == True:
        dmg = dmg*1.5
    #consider weaknesses
    if deftype1 == deftype2:
        dmg = dmg*type_weakness(movetype,deftype1)
    else:
        dmg = dmg*type_weakness(movetype,deftype1)
        dmg = dmg*type_weakness(movetype,deftype2)
    #if crits
    if crit == True:
        dmg = 1.5*dmg
    #weatherball revenge (only in clear weather)
    #if WBR == True:
    #    dmg = dmg*2
    #helping hand
    #if HH == True:
    #    dmg = dmg*1.5
    #electric type charge effect
    #if charge == True:
    #    dmg =dmg*2
    #any other effects; e.g. abilities
    dmg = dmg*other
    return [int(dmg*(85/100)),int(dmg)]

# Base Stats at Level

We use the following function to determine a pokemons base stats at a certain level, this does not include IVs, EVs or natures. However, in the function IVs and EVs can be seen. Natures would multiply one stat by 1.1 and another by 0.9.

In [8]:
def base_stats_at_lvl(Pokemon,level,df):
    pkm = df[df.index == Pokemon]
    IV = 0#[0,0,0,0,0,0]
    EV = 0#[0,0,0,0,0,0]
    HP = ((2*pkm['HP']+IV+(EV/4))*level)/100+level+10
    ATK = ((2*pkm['ATK']+IV+(EV/4))*level)/100+5
    DEF = ((2*pkm['DEF']+IV+(EV/4))*level)/100+5
    SPATK = ((2*pkm['SPATK']+IV+(EV/4))*level)/100+5
    SPDEF = ((2*pkm['SPDEF']+IV+(EV/4))*level)/100+5
    SPD = ((2*pkm['SPD']+IV+(EV/4))*level)/100+5
    return [int(HP),int(ATK),int(DEF),int(SPATK),int(SPDEF),int(SPD)]

# A/D Ratio

We create a function to calculate the A/D ratio between two pokemon. This is the difference in the attack stat with the defence stat or the special attack stat with the special defence stat. (There are some exceptions such as Psyshock and Psystrike).

In [9]:
def phys_spec_ratio(atk_pkm,atk_lvl,def_pkm,def_lvl,df):
    pkm1 = base_stats_at_lvl(atk_pkm,atk_lvl,df)
    pkm2 = base_stats_at_lvl(def_pkm,def_lvl,df)
    phys = pkm1[1]/pkm2[2]
    spec = pkm1[3]/pkm2[4]
    return[float(phys),float(spec)]

# Get Moves (only works for some pokemon)

We convert the code from Serebii move scrape into a useable function. This involves doing the webscraping on the fly.

In [10]:
import bs4 as bs
import urllib.request
def get_moves(pokemon):
    #if two_egg_group == True:
    k = 8
    #else:
    #    k = 9
    sauce = urllib.request.urlopen('https://www.serebii.net/pokedex-swsh/'+ pokemon+ '/').read()
    soup = bs.BeautifulSoup(sauce,'lxml')
    table = soup.find_all('table',{'class': 'dextable'})
    level = []
    move = []
    cat = []
    typ = []
    power = []
    accuracy = []
    pp = []
    eff_percent = []
    flavour = []
    j = k
    #loop through tables collecting data
    while j <= k+3:
        final = []
        for t in table[j].find_all('tr'):
            for i in t.find_all('td'):
                trial = i
                final.append(trial)
        i = 1
        while i < len(final):
            if i%9 == 1:
                level.append(final[i].text)
            if i%9 == 2:
                move.append(final[i].text)
            if i%9 == 3:
                typ.append(final[i])
            if i%9 == 4:
                cat.append(final[i])
            if i%9 == 5:
                power.append(final[i].text)
            if i%9 == 6:
                accuracy.append(final[i].text)
            if i%9 == 7:
                pp.append(final[i].text)
            if i%9 == 8:
                eff_percent.append(final[i].text)
            if i%9 == 0:
                flavour.append(final[i].text)
            i = i+1
        j = j+1
    #create dataframes from TM and level up moves
    df = pd.DataFrame({'MOVE':move})
    df['TYPE'] = typ
    df['CATEGORY'] = cat
    df['POWER'] = power
    df['ACCURACY'] = accuracy
    df['PP'] = pp
    df['EFF %'] = eff_percent
    df['LVL'] = level
    df['INFO'] = flavour
    move = []
    cat = []
    typ = []
    power = []
    accuracy = []
    pp = []
    eff_percent = []
    flavour = []
    #loop through egg moves
    if j == k+4:
        final = []
        for t in table[j].find_all('tr'):
            for i in t.find_all('td'):
                trial = i
                final.append(trial)
        i = 1
        while i < len(final):
            if i%9 == 1:
                move.append(final[i].text)
            if i%9 == 2:
                typ.append(final[i])
            if i%9 == 3:
                cat.append(final[i])
            if i%9 == 4:
                power.append(final[i].text)
            if i%9 == 5:
                accuracy.append(final[i].text)
            if i%9 == 6:
                pp.append(final[i].text)
            if i%9 == 7:
                eff_percent.append(final[i].text)
            if i%9 == 0:
                flavour.append(final[i].text)
            i = i+1
        j = j+1
    #Move Tutor Moves
    if j <= k+6:
        final = []
        for t in table[j].find_all('tr'):
            for i in t.find_all('td'):
                trial = i
                final.append(trial)
        i = 1
        while i < len(final):
            if i%8 == 1:
                move.append(final[i].text)
            if i%8 == 2:
                typ.append(final[i])
            if i%8 == 3:
                cat.append(final[i])
            if i%8 == 4:
                power.append(final[i].text)
            if i%8 == 5:
                accuracy.append(final[i].text)
            if i%8 == 6:
                pp.append(final[i].text)
            if i%8 == 7:
                eff_percent.append(final[i].text)
            if i%8 == 0:
                flavour.append(final[i].text)
            i = i+1
        j = j+1
    #make egg dataframe
    df2 = pd.DataFrame({'MOVE':move})
    df2['TYPE'] = typ
    df2['CATEGORY'] = cat
    df2['POWER'] = power
    df2['ACCURACY'] = accuracy
    df2['PP'] = pp
    df2['EFF %'] = eff_percent
    df2['INFO'] = flavour
    df2['MOVE'] = df2.MOVE.str.replace("SWSH Only", "")
    df2['MOVE'] = df2.MOVE.str.replace("BDSP Only", "")
    #merge dataframes
    frames = [df,df2]
    result = pd.concat(frames)
    result['LVL'].fillna("Egg", inplace=True)
    result['TYPE'] = result['TYPE'].astype(str).replace('.*(?=/type/)','',regex=True).replace('/type/','',regex=True).replace('(?=.gif).*','',regex=True).replace('.gif','',regex=True)
    result['CATEGORY'] = result['CATEGORY'].astype(str).replace('.*(?=/type/)','',regex=True).replace('/type/','',regex=True).replace('(?=.png).*','',regex=True).replace('.png','',regex=True)
    result = result.drop_duplicates(subset=['MOVE'],keep='first') #since the rows are now sorted in descending order
    result = result.set_index('MOVE')
    result['POKEMON'] = pokemon
    return result
get_moves('charizard')

,TYPE,CATEGORY,POWER,ACCURACY,PP,EFF %,LVL,INFO,POKEMON
MOVE,,,,,,,,,
Air Slash,flying,special,75,95,15,30,Evolve,The user attacks with a blade of air that slic...,charizard
Dragon Claw,dragon,physical,80,100,15,--,—,The user slashes the target with huge sharp cl...,charizard
Heat Wave,fire,special,95,90,10,10,—,The user attacks by exhaling hot breath on opp...,charizard
Scratch,normal,physical,40,100,35,--,—,"Hard, pointed, sharp claws rake the target to ...",charizard
Growl,normal,other,--,100,40,--,—,"The user growls in an endearing way, making op...",charizard
...,...,...,...,...,...,...,...,...,...
Wing Attack,flying,physical,60,100,35,--,Egg,"The target is struck with large, imposing wing...",charizard
Dragon Tail,dragon,physical,60,90,10,--,Egg,"The target is knocked away, and a different Po...",charizard
Air Cutter,flying,special,60,95,25,--,Egg,The user launches razor-like wind to slash opp...,charizard


In [11]:
#from tqdm import tnrange, tqdm_notebook
#from urllib.error import HTTPError
#moves = get_moves('bulbasaur')
#for index, row in tqdm_notebook(df.iterrows(),desc='main loop'):
#    try:
#        df1 = get_moves(index.lower().replace(" ", ""))
#        frames = [moves,df1]
#        moves = pd.concat(frames)
#    except ValueError as err:
#        pass
#    except IndexError as err:
#        pass
#    except HTTPError as err:
#        pass
#moves

In [12]:
#moves.drop_duplicates(subset=['POKEMON'],keep='first')

# Function Testing

In [13]:
gen1_dmg(attack=27,defence=16,lvl=15,crit=True,power=40,movetype='normal',physical=True,stab=True,deftype1='normal',deftype2='normal')


[26, 31]

In [14]:
dmg(attack=27,defence=16,lvl=15,crit=True,power=40,movetype='fairy',physical=True,stab=False,deftype1='dark',deftype2='dragon',item=1,weather=False,burn=False,screen=False,double=False,targetall=False,FF=False,WBR=False,HH=False,charge=False,other=1)

TypeError: dmg() got an unexpected keyword argument 'attack'

In [ ]:
type_weakness('fairy','dark')

In [ ]:
base_stats_at_lvl('Patrat',15)

In [ ]:
phys_spec_ratio('Patrat',15,'Snubbull',15)